In [14]:
from bs4 import BeautifulSoup
from wikitools import wiki, api
import json
import re
from collections import defaultdict

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

    
def check_revision(revisions):
    redir=""
    page=False
    for j in revisions:
        redir_marker=["#WEITERLEITUNG","#REDIRECT","#redirect","#weiterleitung","#Weiterleitung","#Redirect"]
        if ("#WEITERLEITUNG" in j["*"])|("#REDIRECT" in j["*"])|("#redirect" in j["*"])|("#weiterleitung" in j["*"])\
        |("#Redirect" in j["*"])|("#Weiterleitung" in j["*"]):
            m = re.search(r"\[\[(.+)\]\]", j["*"])
            s = m.group(0)[2:].split('#', 1)[0].replace("]]",'')
            redir=s
        else:
            page=True
    return redir,page


def check_with_mark_beruf(beruf_m):
    beruf_markers=[" Beruf"," (Beruf)"," beruf", " (Amt)", " Amt"]
    for i in beruf_markers:
        l="de" 
        site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
        params={'action':'query','titles':beruf_m+i,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
        request = api.APIRequest(site, params)
        result=request.query()
        if result["query"]["pages"].values()[0].get("revisions"):
            redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
            if page:
                page=beruf_m+i
            return redir,page     
    return False,False

def check_with_im(beruf_m):
    beruf_markers=[" "," für ".decode('utf-8')]
    for i in beruf_markers:
        l="de" 
        site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
        params={'action':'query','titles':beruf_m.replace(" im ",i),'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
        request = api.APIRequest(site, params)
        result=request.query()
        if result["query"]["pages"].values()[0].get("revisions"):
            redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
            if page:
                page=beruf_m.replace(" im ",i)
            return redir,page     
    return False,False



words=load_simple_json('de/occupation_all.json')

print words[words.keys()[0]]
a=defaultdict(list)
for m in words:
    if "Technischer Konfektion" in m:
        print m
        a[m]= words[m]

#words = {"Lehrer":[[],"Lehrerin"],"Professor":[[],"Professorin"], "Profesghhsor":[[],"Professorin"]}
words={    "Technischer Konfektion\u00e4r": [
        [
            "Technische Konfektion\u00e4rin und Technischer Konfektion\u00e4r", 
            "Technische Konfektion\u00e4rin oder Technischer Konfektion\u00e4r", 
            "Technische Konfektion\u00e4rin bzw. Technischer Konfektion\u00e4r"
        ], 
        "Technische Konfektion\u00e4rin"
    ], 
    "Kapit\u00e4n": [
        [
            "Kapit\u00e4nIn", 
            "Kapit\u00e4nin und Kapit\u00e4n", 
            "Kapit\u00e4nin oder Kapit\u00e4n", 
            "Kapit\u00e4nin bzw. Kapit\u00e4n"
        ], 
        "Kapit\u00e4nin"
    ]}
words=a
redirection_m={}
redirection_f={}
page_m=[]
page_m_new=[]
no_page_m=[]
no_page_m_new=[]
page_f=[]
no_page_f=[]

#=====progressbar
import sys #for progressbar
k=0#for the progressbar
number_pages=len(words)#for the progressbar
steps = number_pages/100 #progressbar step (each 1% show)
#======

for beruf_m in words:
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    #all revisions
    #http://de.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=timestamp|content&rvlimit=500&titles=Professorin&format=json&continue=
    #params={'action':'query','titles':beruf_m,'prop':'revisions','rvprop':'timestamp|content','rvlimit':'500','format':'json','continue':''}
    #last revision
    #http://de.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=timestamp|content&titles=Professorin&format=json&continue=
    
    #try beruf_m
    #print beruf_m
    params={'action':'query','titles':beruf_m,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    
    if result["query"]["pages"].values()[0].get("revisions"):
        redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
        if redir:
            redirection_m[beruf_m]=redir
        if page:
            page_m.append(beruf_m)        
    else:
        #there is no page
        #print "no page",beruf_m
        #redir,page=check_with_mark_beruf(beruf_m)
        if " im " in beruf_m:
            redir,page=check_with_im(beruf_m)
            if redir:
                redirection_m[beruf_m]=redir
            elif page:
                page_m_new.append(page)   ######we addd beruf_m+i, where i is some substring,e.g. " (Beruf)"           
            else:
                no_page_m_new.append(beruf_m)
        else:
            no_page_m_new.append(beruf_m)
        no_page_m.append(beruf_m)
        
    #try beruf_f 
    beruf_f =words[beruf_m][1]
    #print beruf_f
    params={'action':'query','titles':beruf_f,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    if result["query"]["pages"].values()[0].get("revisions"):
        redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
        if redir:
            redirection_f[beruf_f]=redir
        if page:
            page_f.append(beruf_f)         
    else:
        #there is no page
        #print "no page",beruf_f
        no_page_f.append(beruf_f) 
        
    #=====progressbar 
    if steps!=0:
        if k%steps == 0:
            print '\b'+str((k)*100/number_pages)+'%',
            sys.stdout.flush()
    k=k+1#forthe  progressbar

                

[[u'RevisorIn', u'Revisorin und Revisor', u'Revisorin oder Revisor', u'Revisorin bzw. Revisor'], u'Revisorin']
Technischer Konfektionär im Spezialarbeiten
Technischer Konfektionär im Flächenverbindung
Technischer Konfektionär
Technischer Konfektionär im Fertigungsvorbereitung


In [15]:
no_page_m

[u'Technischer Konfektion\xe4r im Fertigungsvorbereitung',
 u'Technischer Konfektion\xe4r im Fl\xe4chenverbindung',
 u'Technischer Konfektion\xe4r im Spezialarbeiten']

In [16]:
page_m

[u'Technischer Konfektion\xe4r']